# 构建检索问答链

在 `C3 搭建数据库` 章节，我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

In [1]:
import sys
sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

# 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.vectorstores.chroma import Chroma

从环境变量中加载你的 API_KEY

In [2]:
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

加载向量数据库，其中包含了 ../../data_base/knowledge_db 下多个文档的 Embedding

In [ ]:
# 定义 Embeddings
embedding = ZhipuAIEmbeddings()

# 向量数据库持久化路径
persist_directory = '../../data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)

/var/folders/yd/c4q_f88j1l70g7_jcb6pdnb80000gn/T/ipykernel_17468/4214663976.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [4]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：1004


我们可以测试一下加载的向量数据库，我们可以通过`as_retriever`方法把向量数据库构造成检索器。我们使用一个问题 query 进行向量检索。如下代码会在向量数据库中根据相似性进行检索，返回前 k 个最相似的文档。

In [5]:
question = "什么是prompt engineering?"
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
docs = retriever.invoke(question)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：3


打印一下检索到的内容

In [6]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

检索到的第0个内容: 
 具体来说，首先编写初版 Prompt，然后通过多轮调整逐步改进，直到生成了满意的结果。对于更复杂的应用，可以在多个样本上进行迭代训练，评估 Prompt 的平均表现。在应用较为成熟后，才需要采用在多个样本集上评估 Prompt 性能的方式来进行细致优化。因为这需要较高的计算资源。

总之，Prompt 工程师的核心是掌握 Prompt 的迭代开发和优化技巧，而非一开始就要求100%完美。通过不断调整试错，最终找到可靠适用的 Prompt 形式才是设计 Prompt 的正确方法。

读者可以在 Jupyter Notebook 上，对本章给出的示例进行实践，修改 Prompt 并观察不同输出，以深入理解 Prompt 迭代优化的过程。这会对进一步开发复杂语言模型应用提供很好的实践准备。

三、英文版

产品说明书
-----------------------------------------------------
检索到的第1个内容: 
 第一章 简介

欢迎来到面向开发者的提示工程部分，本部分内容基于吴恩达老师的《Prompt Engineering for Developer》课程进行编写。《Prompt Engineering for Developer》课程是由吴恩达老师与 OpenAI 技术团队成员 Isa Fulford 老师合作授课，Isa 老师曾开发过受欢迎的 ChatGPT 检索插件，并且在教授 LLM （Large Language Model， 大语言模型）技术在产品中的应用方面做出了很大贡献。她还参与编写了教授人们使用 Prompt 的 OpenAI cookbook。我们希望通过本模块的学习，与大家分享使用提示词开发 LLM 应用的最佳实践和技巧。
-----------------------------------------------------
检索到的第2个内容: 
 第二章 提示原则

如何去使用 Prompt，以充分发挥 LLM 的性能？首先我们需要知道设计 Prompt 的原则，它们是每一个开发者设计 Prompt 所必须知道的基础概念。本章讨论了设计高效 Prompt 的两个关键原则：编写清晰、具体的指令和给予模型充足思考时间。掌握这两点，对创建可靠的语言模型交互尤为重要。

首先，Prom

## 2.创建检索链
我们可以使用LangChain的LCEL(LangChain Expression Language, LangChain表达式语言)来构建workflow，LCEL可以支持异步（ainvoke）、流式(stream)、批次处理(batch)等多种运行方式，同时还可以使用LangSmith无缝跟踪。

接下来我们使用刚才定义的retriever定义一个简单的检索链。

In [7]:
from langchain_core.runnables import RunnableLambda
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

combiner = RunnableLambda(combine_docs)
retrieval_chain = retriever | combiner

retrieval_chain.invoke("南瓜书是什么？")

'前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推\n导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充\n具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；\n• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得\n\n最新版PDF 获取地址：https://github.com/datawhalechina/pumpkin-book/releases\n编委会\n主编：Sm1les、archwalker、jbb0523\n编委：juxiao、Majingmin、MrBigFan、shanry、Ye980226\n封面设计：构思-Sm1les、创作-林王茂盛\n致谢\n特别感谢awyd234、feijuan、Ggmatch、Heitao5200、huaqing89、LongJH、LilRachel、LeoLRH、Nono17、\nspareribs、sunchaothu、StevenLzq 在最早期的时候对南瓜书所做的贡献。\n扫描下方二维码，然后回复关键词“南瓜书”，即可加入“南瓜书读者交流群”\n版权声明\n本作品采用知识共享署名-非商业性使用-相同方式共享4.0 国际许可协议进行许可。\n\n• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得\n有点飘的时候再回来啃都来得及；\n• 每个公式的解析和推导我们都力(zh

LCEL中要求所有的组成元素都是`Runnable`类型，前面我们见过的`ChatModel`、`PromptTemplate`等都是继承自`Runnable`类。上方的`retrieval_chain`是由检索器`retriever`及组合器`combiner`组成的，由`|`符号串连，数据从左向右传递，即问题先被`retriever`检索得到检索结果，再被`combiner`进一步处理并输出。

## 3. 创建LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [8]:
import os 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [9]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

llm.invoke("请你自我介绍一下自己！").content

'当然可以！我是一个由OpenAI开发的人工智能助手，名叫ChatGPT。我擅长处理和生成自然语言文本，可以帮助回答问题、提供信息、协助解决问题以及进行各种对话。我没有个人经历或情感，但我会尽力提供准确和有用的回答。如果你有任何问题或需要帮助的地方，随时可以问我！'

## 4. 构建检索问答链

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。请你在回答的最后说“谢谢你的提问！”。
{context}
问题: {input}
"""
# 将template通过 PromptTemplate 转为可以在LCEL中使用的类型
prompt = PromptTemplate(template=template)

qa_chain = (
    RunnableParallel({"context": retrieval_chain, "input": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

在上边代码中我们把刚才定义的检索链当作子链作为`prompt`的`context`，再使用`RunnablePassthrough`存储用户的问题作为`prompt`的`input`。又因为这两个操作是并行的，所以我们使用`RunnableParallel`来将他们并行运行。

### 4.1检索问答链效果测试

In [11]:
question_1 = "什么是南瓜书？"
question_2 = "Prompt Engineering for Developer是谁写的？"

In [12]:
result = qa_chain.invoke(question_1)
print("大模型+知识库后回答 question_1 的结果：")
print(result)

大模型+知识库后回答 question_1 的结果：
南瓜书是一本对《机器学习》（西瓜书）中较难理解的公式进行解析和补充推导细节的书籍，旨在帮助读者更好地理解机器学习中的数学推导。它以西瓜书的内容为前置知识，适合在遇到推导困难时查阅。南瓜书的目标是帮助读者成为“理工科数学基础扎实点的大二下学生”。谢谢你的提问！


In [13]:
result = qa_chain.invoke(question_2)
print("大模型+知识库后回答 question_2 的结果：")
print(result)

大模型+知识库后回答 question_2 的结果：
《Prompt Engineering for Developer》课程是由吴恩达老师与 OpenAI 技术团队成员 Isa Fulford 合作授课的。谢谢你的提问！


### 4.2 大模型自己回答的效果

In [14]:
llm.invoke(question_1).content

'南瓜书通常是指《深度学习：算法与实现》这本书，因为书的封面是橙色的，看起来像南瓜。这本书由李沐、阿斯顿张、扎卡里·C·立顿和亚历山大·J·斯莫拉编写，主要介绍深度学习的基础知识和实践方法。书中涵盖了深度学习的基本概念、常用模型和算法，并提供了大量的代码示例，帮助读者理解和实现深度学习技术。由于其内容详实且实用性强，南瓜书在深度学习领域的学习者中受到广泛欢迎。'

In [15]:
llm.invoke(question_2).content

'《Prompt Engineering for Developers》是由 Isa Fulford 和 Andrew Ng 合著的一本书。该书旨在帮助开发者更好地理解和应用提示工程（Prompt Engineering）技术，以提高与大型语言模型（如GPT-3）交互的效率和效果。'

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 向检索链添加聊天记录

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 传递聊天记录

在本节中我们将使用 LangChain 中的`ChatPromptTemplate`，即将先前的对话嵌入到语言模型中，使其具有连续对话的能力。 `ChatPromptTemplate`可以接收聊天消息历史记录，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [16]:
from langchain_core.prompts import ChatPromptTemplate

# 问答链的系统prompt
system_prompt = (
    "你是一个问答任务的助手。 "
    "请使用检索到的上下文片段回答这个问题。 "
    "如果你不知道答案就说不知道。 "
    "请使用简洁的话语回答用户。"
    "\n\n"
    "{context}"
)
# 制定prompt template
qa_prompt = ChatPromptTemplate(
    [
        ("system", system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

In [17]:
# 无历史记录
messages = qa_prompt.invoke(
    {
        "input": "南瓜书是什么？",
        "chat_history": [],
        "context": ""
    }
)
for message in messages.messages:
    print(message.content)

你是一个问答任务的助手。 请使用检索到的上下文片段回答这个问题。 如果你不知道答案就说不知道。 请使用简洁的话语回答用户。


南瓜书是什么？


In [18]:
# 有历史记录
messages = qa_prompt.invoke(
    {
        "input": "你可以介绍一下他吗？",
        "chat_history": [
            ("human", "西瓜书是什么？"),
            ("ai", "西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。"),
        ],
        "context": ""
    }
)
for message in messages.messages:
    print(message.content)

你是一个问答任务的助手。 请使用检索到的上下文片段回答这个问题。 如果你不知道答案就说不知道。 请使用简洁的话语回答用户。


西瓜书是什么？
西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。
你可以介绍一下他吗？


### 5.2 带有信息压缩的检索链

因为我们正在搭建的问答链带有支持多轮对话功能，所以与单轮对话的问答链相比会多面临像上方输出结果的问题，即用户最新的对话语义不全，在使用用户问题查询向量数据库时很难检索到相关信息。像上方的“你可以介绍一下他吗？”，其实是“你可以介绍下周志华老师吗？”的意思。为了解决这个问题我们将采取信息压缩的方式，让llm根据历史记录完善用户的问题。

In [19]:
from langchain_core.runnables import RunnableBranch

# 压缩问题的系统 prompt
condense_question_system_template = (
    "请根据聊天记录完善用户最新的问题，"
    "如果用户最新的问题不需要完善则返回用户的问题。"
    )
# 构造 压缩问题的 prompt template
condense_question_prompt = ChatPromptTemplate([
        ("system", condense_question_system_template),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ])
# 构造检索文档的链
# RunnableBranch 会根据条件选择要运行的分支
retrieve_docs = RunnableBranch(
    # 分支 1: 若聊天记录中没有 chat_history 则直接使用用户问题查询向量数据库
    (lambda x: not x.get("chat_history", False), (lambda x: x["input"]) | retriever, ),
    # 分支 2 : 若聊天记录中有 chat_history 则先让 llm 根据聊天记录完善问题再查询向量数据库
    condense_question_prompt | llm | StrOutputParser() | retriever,
)

## 5.3 支持聊天记录的检索问答链

这里我们使用之前定义的问答模板即`qa_prompt`构造问答链，另外我们通过`RunnablePassthrough.assign`将中间的查询结果存为`"context"`，将最终结果存为`"answer"`。因为查询结果被存为`"context"`，所以我们整合查询结果的函数`combine_docs`也要做相应的改动。

In [20]:
# 重新定义 combine_docs
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs["context"]) # 将 docs 改为 docs["context"]
# 定义问答链
qa_chain = (
    RunnablePassthrough.assign(context=combine_docs) # 使用 combine_docs 函数整合 qa_prompt 中的 context
    | qa_prompt # 问答模板
    | llm
    | StrOutputParser() # 规定输出的格式为 str
)
# 定义带有历史记录的问答链
qa_history_chain = RunnablePassthrough.assign(
    context = (lambda x: x) | retrieve_docs # 将查询结果存为 content
    ).assign(answer=qa_chain) # 将最终结果存为 answer

## 5.4 测试检索问答链

In [21]:
# 不带聊天记录
qa_history_chain.invoke({
    "input": "西瓜书是什么？",
    "chat_history": []
})

{'input': '西瓜书是什么？',
 'chat_history': [],
 'context': [Document(metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': '../../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 1, 'producer': 'xdvipdfmx (20200315)', 'source': '../../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推\n导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充\n具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；\n• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得')

In [22]:
# 带聊天记录
qa_history_chain.invoke({
    "input": "南瓜书跟它有什么关系？",
    "chat_history": [
        ("human", "西瓜书是什么？"),
        ("ai", "西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。"),
    ]
})

{'input': '南瓜书跟它有什么关系？',
 'chat_history': [('human', '西瓜书是什么？'),
  ('ai', '西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。')],
 'context': [Document(metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': '../../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 1, 'producer': 'xdvipdfmx (20200315)', 'source': '../../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推\n导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充\n具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己

可以看到，LLM 准确地判断了“它”是什么，代表着我们成功地传递给了它历史信息。另外召回的内容也有着问题的答案，证明我们的信息压缩策略也起到了作用。这种关联前后问题及压缩信息并检索的能力，可大大增强问答系统的连续性和智能水平。